# Transformation Silver → Gold

## 📋 Objectif
Créer un modèle dimensionnel (star schema) optimisé pour l'analyse.

## 🌟 Modèle créé
- **4 dimensions** : Date, Time, Turbine, Operational Status
- **1 fait** : Production éolienne

## 📦 Dépendances
- **Input** : LH_Wind_Power_Silver.dbo.wind_power
- **Output** : LH_Wind_Power_Gold (5 tables)


In [14]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

StatementMeta(, f3fd345e-a533-42e7-b9e2-b71bbff6c427, 16, Finished, Available, Finished, False)

Charger depuis Silver

In [15]:
silver_table_path = "abfss://Arailym_WindPowerAnalytics@onelake.dfs.fabric.microsoft.com/LH_Wind_Power_Silver.Lakehouse/Tables/dbo/wind_power"

df = spark.read.format("delta").load(silver_table_path)

print(f"📊 Données chargées : {df.count()} lignes")

StatementMeta(, f3fd345e-a533-42e7-b9e2-b71bbff6c427, 17, Finished, Available, Finished, False)

📊 Données chargées : 1296 lignes


dim_date

In [16]:
# Créer dim_date avec valeurs uniques
date_dim = (df
    .select("date", "day", "month", "quarter", "year")
    .distinct()
    .withColumnRenamed("date", "date_id")
)

print(f"📅 dim_date : {date_dim.count()} dates uniques")
date_dim.show(5)

StatementMeta(, f3fd345e-a533-42e7-b9e2-b71bbff6c427, 18, Finished, Available, Finished, False)

📅 dim_date : 3 dates uniques
+-------------------+---+-----+-------+----+
|            date_id|day|month|quarter|year|
+-------------------+---+-----+-------+----+
|2024-06-17 00:00:00| 17|    6|      2|2024|
|2024-06-16 00:00:00| 16|    6|      2|2024|
|2024-06-15 00:00:00| 15|    6|      2|2024|
+-------------------+---+-----+-------+----+



dim_time

In [17]:
# Créer dim_time avec valeurs uniques
time_dim = (df
    .select("time", "hour_of_day", "minute_of_hour", "second_of_minute", "time_period")
    .distinct()
    .withColumnRenamed("time", "time_id")
)

print(f"⏰ dim_time : {time_dim.count()} temps uniques")
time_dim.show(5)

StatementMeta(, f3fd345e-a533-42e7-b9e2-b71bbff6c427, 19, Finished, Available, Finished, False)

⏰ dim_time : 144 temps uniques
+--------+-----------+--------------+----------------+-----------+
| time_id|hour_of_day|minute_of_hour|second_of_minute|time_period|
+--------+-----------+--------------+----------------+-----------+
|10:10:00|         10|            10|               0|    Morning|
|11:10:00|         11|            10|               0|    Morning|
|18:10:00|         18|            10|               0|    Evening|
|18:30:00|         18|            30|               0|    Evening|
|19:10:00|         19|            10|               0|    Evening|
+--------+-----------+--------------+----------------+-----------+
only showing top 5 rows



dim_turbine (avec surrogate key)

In [18]:
# Créer dim_turbine avec ID auto-incrémenté
turbine_dim = (df
    .select("turbine_name", "capacity", "location_name", "latitude", "longitude", "region")
    .distinct()
    .withColumn("turbine_id", 
        row_number().over(
            Window.orderBy("turbine_name", "capacity", "location_name")
        )
    )
)

print(f"🌬️ dim_turbine : {turbine_dim.count()} turbines")
turbine_dim.show()

StatementMeta(, f3fd345e-a533-42e7-b9e2-b71bbff6c427, 20, Finished, Available, Finished, False)

🌬️ dim_turbine : 3 turbines
+------------+--------+-------------+--------+---------+--------+----------+
|turbine_name|capacity|location_name|latitude|longitude|  region|turbine_id|
+------------+--------+-------------+--------+---------+--------+----------+
|   Turbine A|    2200|   Location 1| 34.0522|-118.2437|Region A|         1|
|   Turbine B|    2000|   Location 2| 36.7783|-119.4179|Region B|         2|
|   Turbine C|    2500|   Location 3| 40.7128|  -74.006|Region C|         3|
+------------+--------+-------------+--------+---------+--------+----------+



dim_operational_status


In [19]:
# Créer dim_operational_status avec ID
operational_status_dim = (df
    .select("status", "responsible_department")
    .distinct()
    .withColumn("status_id", 
        row_number().over(
            Window.orderBy("status", "responsible_department")
        )
    )
)

print(f"📊 dim_operational_status : {operational_status_dim.count()} statuts")
operational_status_dim.show()

StatementMeta(, f3fd345e-a533-42e7-b9e2-b71bbff6c427, 21, Finished, Available, Finished, False)

📊 dim_operational_status : 6 statuts
+--------------------+----------------------+---------+
|              status|responsible_department|status_id|
+--------------------+----------------------+---------+
|               Fault|           Engineering|        1|
|          Inspection|           Maintenance|        2|
|             Offline|            Operations|        3|
|              Online|            Operations|        4|
|Preventive Mainte...|           Maintenance|        5|
|Reactive Maintenance|           Maintenance|        6|
+--------------------+----------------------+---------+



Joindre les dimensions

In [20]:
# Joindre pour récupérer les clés étrangères
df_with_keys = (df
    .join(turbine_dim, 
          ["turbine_name", "capacity", "location_name", "latitude", "longitude", "region"], 
          "left")
    .join(operational_status_dim, 
          ["status", "responsible_department"], 
          "left")
)

StatementMeta(, f3fd345e-a533-42e7-b9e2-b71bbff6c427, 22, Finished, Available, Finished, False)

Créer fact_wind_power

In [21]:
# Table de faits avec uniquement clés et mesures
fact_table = (df_with_keys
    .select(
        "production_id",
        col("date").alias("date_id"),
        col("time").alias("time_id"),
        "turbine_id",
        "status_id",
        "wind_speed",
        "wind_direction",
        "energy_produced"
    )
)

print(f"📈 fact_wind_power : {fact_table.count()} mesures")
fact_table.show(10)

StatementMeta(, f3fd345e-a533-42e7-b9e2-b71bbff6c427, 23, Finished, Available, Finished, False)

📈 fact_wind_power : 1296 mesures
+-------------+-------------------+--------+----------+---------+----------+--------------+---------------+
|production_id|            date_id| time_id|turbine_id|status_id|wind_speed|wind_direction|energy_produced|
+-------------+-------------------+--------+----------+---------+----------+--------------+---------------+
|         7291|2024-06-17 00:00:00|21:00:00|         1|        4|     16.64|            SW|        1810.38|
|         7292|2024-06-17 00:00:00|21:00:00|         2|        4|     14.03|             E|        1409.71|
|         7293|2024-06-17 00:00:00|21:00:00|         3|        4|       6.9|             W|        1895.09|
|         7294|2024-06-17 00:00:00|21:10:00|         1|        4|      22.6|             S|        1975.88|
|         7295|2024-06-17 00:00:00|21:10:00|         2|        4|     24.82|             N|         1469.8|
|         7296|2024-06-17 00:00:00|21:10:00|         3|        4|      6.44|            SW|        1521

Sauvegarder dans Gold. Chemins Gold

In [22]:
base_path = "abfss://Arailym_WindPowerAnalytics@onelake.dfs.fabric.microsoft.com/LH_Wind_Power_Gold.Lakehouse/Tables/dbo"

gold_date_dim_path = f"{base_path}/dim_date"
gold_time_dim_path = f"{base_path}/dim_time"
gold_turbine_dim_path = f"{base_path}/dim_turbine"
gold_operational_status_dim_path = f"{base_path}/dim_operational_status"
gold_fact_table_path = f"{base_path}/fact_wind_power"

StatementMeta(, f3fd345e-a533-42e7-b9e2-b71bbff6c427, 24, Finished, Available, Finished, False)

Sauvegarde

In [23]:
# Sauvegarder toutes les tables
date_dim.write.format("delta").mode("overwrite").save(gold_date_dim_path)
time_dim.write.format("delta").mode("overwrite").save(gold_time_dim_path)
turbine_dim.write.format("delta").mode("overwrite").save(gold_turbine_dim_path)
operational_status_dim.write.format("delta").mode("overwrite").save(gold_operational_status_dim_path)
fact_table.write.format("delta").mode("overwrite").save(gold_fact_table_path)

print("✅ Toutes les tables créées dans Gold !")

StatementMeta(, f3fd345e-a533-42e7-b9e2-b71bbff6c427, 25, Finished, Available, Finished, False)

✅ Toutes les tables créées dans Gold !


Intégrité référentielle. verification

In [25]:
from pyspark.sql.functions import count, when

print("=== 🔍 Vérification intégrité référentielle ===\n")

# Vérifier les dates
fact_dates = fact_table.select("date_id").distinct().count()
dim_dates = date_dim.count()
print(f"📅 Dates - Fait: {fact_dates}, Dim: {dim_dates}")

# Vérifier les turbines
fact_turbines = fact_table.select("turbine_id").distinct().count()
dim_turbines = turbine_dim.count()
print(f"🌬️ Turbines - Fait: {fact_turbines}, Dim: {dim_turbines}")

# Vérifier les clés nulles
null_check = fact_table.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in ["date_id", "time_id", "turbine_id", "status_id"]
])
print("\n❌ Clés nulles dans fact :")
null_check.show()

StatementMeta(, f3fd345e-a533-42e7-b9e2-b71bbff6c427, 27, Finished, Available, Finished, False)

=== 🔍 Vérification intégrité référentielle ===

📅 Dates - Fait: 3, Dim: 3
🌬️ Turbines - Fait: 3, Dim: 3

❌ Clés nulles dans fact :
+-------+-------+----------+---------+
|date_id|time_id|turbine_id|status_id|
+-------+-------+----------+---------+
|      0|      0|         0|        0|
+-------+-------+----------+---------+

